In [1]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd 
import re
import math
import os
import sys
import cv2
import glob
import time
import imutils
import numpy as np
#from google.colab.patches import cv2_imshow

In [3]:
path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'

# Cut AIHub Only Violence Part

In [4]:
mp4_files = glob.glob(path+'**/*.mp4', recursive=True)

In [5]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

In [6]:
def aihub_data_cut_violence(video_path, csv_path):
    mp4_files = glob.glob(video_path+'**/*.mp4', recursive=True)
    events = pd.read_csv(csv_path)
    out_path_list = list()

    for idx, i in enumerate(mp4_files):
        start = 0
        end = 0
        for j in events.itertuples():
            if (i.split('AIHub/')[1].split('_resized')[0] == j.fname.split('.')[0]):
                if (len(j.start.split(':')) == 3):
                    start = float(j.start.split(':')[0]) * 60 * 60 + float(j.start.split(':')[1]) * 60 + float(j.start.split(':')[2])
                    end = start + float(j.end.split(':')[0]) * 60 * 60 + float(j.end.split(':')[1]) * 60 + float(j.end.split(':')[2])
                else:
                    start = float(j.start.split(':')[0]) * 60 + float(j.start.split(':')[1]) 
                    end = start + float(j.end.split(':')[0]) * 60 + float(j.end.split(':')[1])
                break
        
        out = i.split('.')[0]+'_fi.mp4'
        # print(i,start,end,out)
        ffmpeg_extract_subclip(i, start, end, targetname=out)
        out_path_list.append(out)
        print('working... {}', round(idx/len(mp4_files),2))
    return out_path_list

#path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#csv_path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/events_time.csv'
#out_path = aihub_data_cut_violence(path, csv_path)

# Box Crop Video


In [7]:
def box_crop_video_by_frame(video_path, out_path, box_width, box_height, object_position):
    '''
    video_path : input video path
    out_path : path for saving video
    box_width : width of box
    box_height : height of box
    object_position : list of object posisitons
    '''

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 영상의 넓이(가로) 프레임
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 영상의 높이(세로) 프레임
    frameRate = int(cap.get(cv2.CAP_PROP_FPS)) 
    top,bottom,left,right = get_box_size(box_width, box_height, width, height, object_position) 
    
    # 비디오 저장 변수
    writer = None
    
    while True:
        # ret : 성공적으로 불러왔는지 확인
        # frame : 읽어온 frame 정보    
        ret, frame = cap.read()
        
        # 읽은 Frame이 없는 경우 종료
        if not ret:
            break

        # Frame Resize
        frame = frame[top:bottom,left:right,:]
        #cv2.imshow('frame', frame)
        #cv2_imshow(frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        
        # 저장할 video 설정
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MJPG
            writer = cv2.VideoWriter(out_path, fourcc, frameRate, (frame.shape[1], frame.shape[0]), True)
        
        # 비디오 저장
        if writer is not None:
            writer.write(frame)
            
    print('Successfully cropped : ', out_path)
    return out_path

In [17]:
def get_box_size(box_width, box_height, vid_width, vid_height, people):
    '''
    box_width : width of the box
    box_height : height of the box
    vid_width : width of input video
    vid_height :  height of input video
    people : list of object position [[x1,y1],[x2,y2], ... [xn,yn]]]

    returns cornor coordinate of box in the video
    '''
    print('cutting {}x{} in {}x{} video'.format(box_width, box_height, vid_width, vid_height))
    pos_wh = np.array([[x[0],x[1]] for x in people])
    center = pos_wh.mean(axis=0)
    print('center :', center)
    left = int(center[0] - box_width/2)
    top = int(center[1] -  box_height/2)
    if left > vid_width - box_width:
        left = vid_width - box_width 
    elif left < 0:
        left = 0

    if top > vid_height - box_height:
        top = vid_height - box_height 
    elif top < 0:
        top = 0

    right = left + box_width
    bottom = top + box_height

    print('box pixel area : {}, {}'.format((left,top),(right,bottom)))
    assert left<right and top<bottom, 'left, top has to be lower than right, bottom'
    assert right <= vid_width and bottom<=vid_height, 'box goes out of the video'
    return top,bottom,left,right

In [9]:
aihub_pos = pd.read_csv('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/aihub_position.csv')
aihub_info = pd.read_csv('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/aihub_info.csv')

In [10]:
aihub_info.shape, aihub_pos.shape

((1283, 8), (2830, 4))

In [11]:
aihub_info.drop(labels=['width','height', 'depth','population'], axis=1, inplace=True)

In [12]:
aihub_info.head()

,filename,eventname,start,duration
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,assault,90,200
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,assault,51,248
2,outsidedoor_01/11-2/11-2_cam01_assault01_place...,assault,92,200
3,outsidedoor_01/11-2/11-2_cam01_assault01_place...,assault,52,248
4,outsidedoor_01/12-3/12-3_cam02_assault01_place...,assault,81,163


In [13]:
aihub_pos.head()

,filename,objname,x,y
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,2750,1325
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,3323,1164
2,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,2926,1473
3,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,3509,1146
4,outsidedoor_01/11-2/11-2_cam01_assault01_place...,person_1,721,1063


In [14]:
# video was downscaled from 3840x2160 to 720x406
aihub_pos['x'] = (aihub_pos['x']/3840*720).map(lambda x: int(x))
aihub_pos['y'] = (aihub_pos['y']/2160*406).map(lambda x: int(x))

In [15]:
aihub_pos.columns = ['filename','objname','width','height']

In [16]:
aihub_pos.head()

,filename,objname,width,height
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,515,249
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,623,218
2,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,548,276
3,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,657,215
4,outsidedoor_01/11-2/11-2_cam01_assault01_place...,person_1,135,199


In [19]:
def ffmpeg_extract_box_cropped_subclip(filename, box_width, box_height,left,top, targetname=None):
    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-i", filename,
           "-vf", "crop={}:{}:{}:{}".format(box_width,box_height,left,top),
           targetname]

    subprocess_call(cmd)

In [21]:
def box_crop(path, pos_df, box_width, box_height):
    '''
    works same as box_crop_ffmpeg but this function use cv2 to crop and this takes longer time
    SO DONT USE
    '''
    fight_vid_list = glob.glob(path + '**/*_resized_720_fi.mp4', recursive=True)
    cropped_video = list()

    for idx, in_path in enumerate(fight_vid_list):
        filename = in_path.split(path)[-1].split('_resized_720_fi.mp4')[0]

        obj_pos = list()
        for index, row in pos_df[pos_df['filename']==filename + '.xml'].iterrows():
            obj_pos.append([row.width, row.height])

        out_path = in_path.split('_fi.mp4')[0] + '_cropped_fi.mp4'
        print('working on : ',filename)
        cropped_video.append(box_crop_video_by_frame(in_path, out_path, box_width, box_height, obj_pos))
        print('---- working... {}% finished !!'.format(round(idx/len(fight_vid_list),2)))
        
    return cropped_video

path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#cropped_vid = box_crop(path, aihub_pos, 360, 360)

In [24]:
# cropped_fi.mp4 파일 제거
key = str(input('Are you sure to delete all _fi.mp4 files?\ntype "yes" if so..\n> '))
if key == "yes":
    for vid in glob.glob(path + '**/*_cropped_fi.mp4', recursive=True):
        if os.path.isfile(vid):
            os.remove(vid)
        else:
            print(vid,' doesnt\'t exist')
    print('done')

Are you sure to delete all _fi.mp4 files?
type "yes" if so..
> no


In [25]:
# _fi.mp4 파일 제거
key = str(input('Are you sure to delete all _fi.mp4 files?\ntype "yes" if so..\n> '))
if key == "yes":
    for vid in glob.glob(path + '**/*_cropped_fi.mp4', recursive=True):
        if os.path.isfile(vid):
            os.remove(vid)
        else:
            print(vid,' doesnt\'t exist')
    print('done')

Are you sure to delete all _fi.mp4 files?
type "yes" if so..
> no


In [22]:
aihub_df = pd.merge(aihub_pos, aihub_info, left_on='filename', right_on='filename', how='outer')

In [18]:
def box_crop_ffmpeg(path, pos_df, box_width, box_height):
    fight_vid_list = glob.glob(path + '**/*_resized_720_fi.mp4', recursive=True)
    cropped_video = list()

    for idx, in_path in enumerate(fight_vid_list):
        filename = in_path.split(path)[-1].split('_resized_720_fi.mp4')[0]
        out_path = in_path.split('_fi.mp4')[0] + '_cropped_fi.mp4'

        if os.path.isfile(out_path):
            continue

        obj_pos = list()
        start_time = time.time()
        start = 0
        end = 0
        
        for index, row in pos_df[pos_df['filename']==filename + '.xml'].iterrows():
            if not start or not end:
                start = row.start
                end = start + row.duration
            obj_pos.append([row.width, row.height])
        print(start, end)

        print('working on : ',filename)
        cap = cv2.VideoCapture(in_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 영상의 넓이(가로) 프레임
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 영상의 높이(세로) 프레임
        frameRate = int(cap.get(cv2.CAP_PROP_FPS)) 
        top,_,left,_ = get_box_size(box_width, box_height, width, height, obj_pos)         
        cap.release()

        ffmpeg_extract_box_cropped_subclip(in_path,box_width, box_height,left,top,out_path)
        cropped_video.append(out_path)
        print('---- working... {}%... {} out of {} finished !!\ntime took : {} sec'.format(round(idx+1/len(fight_vid_list) * 100,2), idx+1, len(fight_vid_list), round(time.time()-start_time,3)))

    return cropped_video

#path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#box_cropped_vid = box_crop_ffmpeg(path, aihub_df, 360, 360)

# non-fight data subclip extract

In [6]:
file1 = 'C:/Users/Seogki/Desktop/AIHub 제외했던 vgg19+bilstm.mp4'

In [8]:
ffmpeg_extract_subclip(file1, 0,78,file1.split('.')[0] + '_1.mp4')

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
